In [1]:
import asyncio
import os
import re
import json
from copy import copy, deepcopy
from pathlib import Path
from pprint import pprint, PrettyPrinter
from time import time, sleep
from typing import List, Dict
from uuid import uuid4
from collections import defaultdict

import evaluate
import openai
import requests
import tiktoken
from bs4 import BeautifulSoup, Comment
from doctran import Doctran, ExtractProperty
from dotenv import load_dotenv, find_dotenv
from evaluate import load
from langchain.chat_models import ChatOpenAI
from langchain.docstore.document import Document
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import (
    OpenAIEmbeddings,
    HuggingFaceEmbeddings,
)
from langchain.llms import OpenAI
from langchain.text_splitter import (
    MarkdownTextSplitter,
    MarkdownHeaderTextSplitter,
    LineType,
    RecursiveCharacterTextSplitter,
)
from loguru import logger
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

from main import (
    extract_plan_and_content_wikipedia,
    compare_documents_content,
    compare_documents_sections,
    extract_plan_and_content_patent,
)

_ = load_dotenv(find_dotenv())
openai.api_key = os.getenv('OPENAI_API_KEY')

llm_default = ChatOpenAI(model_name="gpt-3.5-turbo", streaming=True)
llm_16k = ChatOpenAI(model_name="gpt-3.5-turbo-16k", streaming=True)

def num_tokens_from_string(string: str, encoding_name: str = "gpt-3.5-turbo") -> int:
    """Returns the number of tokens in a text string."""
    try:
        encoding = tiktoken.get_encoding(encoding_name)
    except ValueError:
        encoding = tiktoken.encoding_for_model(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

def convert_to_markdown(article_dict):
    md_text = ""

    for heading, content in article_dict.items():
        # heading is of form: 'h3 Example'
        # Define the markdown equivalent for the heading level
        heading_level = "#" * int(heading[1])
        heading = heading[3:]
        # Append the heading and the content to the markdown text
        md_text += f"{heading_level} {heading}\n\n{content}\n\n"

    return md_text


def truncated_pprint(obj, N=5):
    """Pretty print an object, truncating lists and strings to N items/characters
    for easier viewing of plan_json objects"""
    def truncate(item, N):
        if isinstance(item, list) and N is not None:
            return item[:N] + (['...'] if len(item) > N else [])
        if isinstance(item, str) and N is not None:
            N = 125
            return item[:N] + ('...' if len(item) > N else '')
        return item

    def trunc_recursive(item, N):
        if isinstance(item, list):
            return [trunc_recursive(i, N) for i in truncate(item, N)]
        elif isinstance(item, dict):
            return {k: trunc_recursive(v, N) for k, v in item.items()}
        else:
            return truncate(item, N)

    truncated_obj = trunc_recursive(obj, N)
    pprint(truncated_obj, sort_dicts=False)

# Test
data = {
    'long_list': list(range(100)),
    'long_string': 'a' * 100,
    'nested': {
        'nested_list': list(range(50)),
        'nested_string': 'b' * 50
    }
}

truncated_pprint(data, 5)


{'long_list': [0, 1, 2, 3, 4, '...'],
 'long_string': 'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa',
 'nested': {'nested_list': [0, 1, 2, 3, 4, '...'],
            'nested_string': 'bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbb'}}


In [2]:
plan_json = await extract_plan_and_content_patent('data/patents/CONDOM FOR ORAL-GENITAL USE.txt')

2023-08-09 18:23:16.911 | INFO     | main:extract_plan_and_content_patent:679 - 
	Extracting plan and content for patent: data/patents/CONDOM FOR ORAL-GENITAL USE.txt
2023-08-09 18:23:16.912 | INFO     | main:load_patent_file:81 - Loading patent file: data/patents/CONDOM FOR ORAL-GENITAL USE.txt
2023-08-09 18:23:16.913 | INFO     | main:divide_sections_if_too_large:215 - Dividing sections if too large in plan and section content.
2023-08-09 18:23:18.782 | INFO     | main:divide_sections_if_too_large:268 - Added 'EP 0600858 B1' split original heading 'descr'
2023-08-09 18:23:19.826 | INFO     | main:divide_sections_if_too_large:268 - Added 'Male Condom and Protection during Oral Sex' split original heading 'descr'
2023-08-09 18:23:20.688 | INFO     | main:divide_sections_if_too_large:268 - Added 'Condoms for Safe Oral Sex' split original heading 'descr'
2023-08-09 18:23:21.673 | INFO     | main:divide_sections_if_too_large:268 - Added 'Condom for Oral-Genital Use' split original heading

In [3]:
truncated_pprint(plan_json)

{'id': 'c6404bdf-872d-40f8-977d-f57c1cc8161c',
 'title': 'CONDOM FOR ORAL-GENITAL USE',
 'abstract': 'EP\t0600858\tB1\t2000-05-17\ten\tDESCR\t1\t<heading '
             'id="h0001"><u><b>Background of the Invention</b></u></heading><p '
             'id="p0001" num=...',
 'title_embedding_1': [-0.01705644652247429,
                       0.014407929964363575,
                       0.009422097355127335,
                       -0.006363061256706715,
                       0.0016371143283322453,
                       '...'],
 'title_embedding_2': [-0.020731359720230103,
                       0.005263666156679392,
                       -0.05795208364725113,
                       -0.01253290381282568,
                       0.0484771654009819,
                       '...'],
 'abstract_embedding_1': [-0.02011685259640217,
                          0.004784218035638332,
                          0.004644706379622221,
                          -0.01600637286901474,
                       

In [8]:
Path(f'{plan_json["title"]}.json').absolute()

WindowsPath('C:/Users/User/Documents/1 Projects/document_extraction/CONDOM FOR ORAL-GENITAL USE.json')

In [4]:
with open(f'{plan_json["title"]}.json', 'w') as file:
    json.dump(plan_json, file, indent=4)

In [5]:
plan_json['title']

'CONDOM FOR ORAL-GENITAL USE'

In [7]:
with open('condom.json', 'r') as file:
    loaded = json.load(file)

loaded.keys()

dict_keys(['id', 'title', 'abstract', 'title_embedding_1', 'title_embedding_2', 'abstract_embedding_1', 'abstract_embedding_2', 'plan', 'plan_embedding_1', 'plan_embedding_2', 'embedding1_model', 'embedding2_model', 'success', 'error'])

In [4]:
with open('data/2022week30_EP0600000_extract.txt', 'r') as f:
    text = f.read()

with open('data/2022week30_EP0600000_extract.txt', 'r') as f:
    lines = f.readlines()

In [5]:
len(lines)

200

* \ten\t shows it's English
* \tfr\t shows it's French etc.
* Contains TITLE, DESCR, and CLAIM
* Sometimes CLAIM 1 and CLAIM 2 are written in different languages. Do we still need to translate them? No. Even though it says 'CLAIM 2' it's actually the same information, just take the english bit.

Results
* Only ever 1 x CLAIM
* Split on TITLE and everything is fine.

In [6]:
claim_en = lines[20]
claim_de = lines[21]
claim_fr = lines[22]
claim_de

'EP\t0668908\tB1\t2000-05-03\tde\tCLAIM\t2\t<claim id="c-de-01-0001" num="0001"><claim-text>Verfahren zur differenzierten Diagnose einer NematodenInfektion mit einem speziellen Nematoden-Typ, das im Fall von gemischten Nematoden-Infektionen anwendbar ist, wobei bei dem Verfahren eine DNA-Sequenz, die das Î²-Tubulin-Gen oder ein Teil dieses Gens dieser Nematode ist, als Oligonukleotid-Sonde und/oder Primer verwendet wird, und der Teil mindestens 10, vorzugsweise mindestens 18 Nukleotide, umfaÃŸt.</claim-text></claim><claim id="c-de-01-0002" num="0002"><claim-text>Verfahren nach Anspruch 1, bei dem die spezielle Nematode Haemonchus contortus und das Î²-Tubulin-Gen das Î²-Tubulin-Gen von Haemonchus contortus ist.</claim-text></claim><claim id="c-de-01-0003" num="0003"><claim-text>Verfahren nach Anspruch 1 oder 2, bei dem die Sonde und/oder der Primer einen Teil der 3\'-HÃ¤lfte des Î²-Tubulin-Gens umfaÃŸt.</claim-text></claim><claim id="c-de-01-0004" num="0004"><claim-text>Verfahren nach e

In [7]:
claim_en

'EP\t0668908\tB1\t2000-05-03\ten\tCLAIM\t1\t<claim id="c-en-01-0001" num="0001"><claim-text>A method for differentiated diagnosis of nematode infection with a specific type of nematode useful in the case of mixed nematode infections said method comprising use of a DNA sequence, said DNA sequence being the Î²-tubulin gene or a part of said gene of said nematode as oligonucleotide probe and/or primer, said part comprising at least 10, preferably at least 18 nucleotides.</claim-text></claim><claim id="c-en-01-0002" num="0002"><claim-text>A method according to claim 1 wherein the specific nematode is <u>Haemonchus</u> <u>contortus</u> and the Î²-tubulin gene is the Î²-tubulin gene of <u>Haemonchus</u> <u>contortus</u>.</claim-text></claim><claim id="c-en-01-0003" num="0003"><claim-text>A method according to claim 1 or 2 wherein the probe and/or primer comprises a part of the 3\' half of the Î²-tubulin gene.</claim-text></claim><claim id="c-en-01-0004" num="0004"><claim-text>A method accord

In [8]:
lines_en = [line for line in lines if '\ten\t' in line]
lines_en[0]

'EP\t0600858\tB1\t2000-05-17\ten\tTITLE\t1\tCONDOM FOR ORAL-GENITAL USE\n'

In [9]:
lines_en[4]

'EP\t0600865\tB1\t2000-09-20\ten\tTITLE\t1\tXYLANASE PRODUCTION\n'

In [10]:
a = defaultdict(str)
a[1] += 'hello'
a

defaultdict(str, {1: 'hello'})

In [11]:
def split_patents_into_individual_files(patents_file):
    """Read in a file containing many patents. Split each patent into its own file, keeping
    only the english parts, and write to disk."""
    # Read in file
    with open(patents_file, 'r') as f:
        lines = f.readlines()
    # Get all eng sections
    lines_en = [line for line in lines if '\ten\t' in line]
    # Split each on TITLE and write to its own file with TITLE as filename
    os.makedirs('data/patents', exist_ok=True)
    title = 'no title found'
    # Create dict of patents
    patents: defaultdict = defaultdict(str)
    for i, x in enumerate(lines_en):
        if '\tTITLE\t' in x:
            title = x.split('\t')[-1].strip()
        patents[title] += x

    # Write each patent to its own file
    for title, content in patents.items():
        filename_friendly_title = "".join(i for i in title if i not in "\/:*?<>|")
        with open(f'data/patents/{filename_friendly_title}.txt', 'w') as f:
            f.write(content)
            logger.info(f'Wrote file: {filename_friendly_title}.txt')

def load_patent_file(patent_file) -> Dict[str, str]:
    """Read in a patent file and return a dict with keys as section titles and values the content.

    Parameters
    ----------
    patent_file : str
        Path to the patent file.

    Returns
    -------
    patent_dict : dict
        Dict with keys as section titles and values the content. Keys are ['title',
        'descr', 'claim_1', 'claim_2', ..., 'claim_n', 'pdfep']. Not all patents
        will have all keys. All will have 'title' at a minimum.
    """
    # Read file
    with open(patent_file, 'r') as f:
        lines: list = f.readlines()

    # Get all english sections
    lines_en: list = [line for line in lines if '\ten\t' in line]

    # Convert into dict with keys as section titles and values the content
    patent_dict = {}
    total_claims = 1
    for x in lines_en:
        if '\tTITLE\t' in x:
            patent_dict['title'] = x
        elif '\tDESCR\t' in x:
            patent_dict['descr'] = x
        elif '\tCLAIM\t' in x:
            # Some patents have multiple claims, so we need to number them
            patent_dict[f'claim_{total_claims}'] = x
            total_claims += 1
        elif '\tPDFEP' in x:
            patent_dict['pdfep'] = x
        else:
            raise ValueError(f"Expected sections in [TITLE, DESCR, CLAIM, PDFEP]. Received: {x}")

    return patent_dict


In [12]:
a = load_patent_file('data/patents/XYLANASE PRODUCTION.txt')
a.keys()

dict_keys(['title', 'descr', 'claim_1', 'pdfep'])

In [13]:
patents: defaultdict = defaultdict(list)
total = 0
for i, x in enumerate(lines_en):
    if '\tTITLE\t' in x:
        print()
        total += 1
        print(f"Total Patents: {total}")
    print(rf"{i}/{len(lines_en)} - {repr(x[:125])}")
    patents[f"patent_{total}"].append(x)


Total Patents: 1
0/93 - 'EP\t0600858\tB1\t2000-05-17\ten\tTITLE\t1\tCONDOM FOR ORAL-GENITAL USE\n'
1/93 - 'EP\t0600858\tB1\t2000-05-17\ten\tDESCR\t1\t<heading id="h0001"><u><b>Background of the Invention</b></u></heading><p id="p0001" num='
2/93 - 'EP\t0600858\tB1\t2000-05-17\ten\tCLAIM\t1\t<claim id="c-en-01-0001" num="0001"><claim-text>A condom (31) suitable for oral-genital u'
3/93 - 'EP\t0600858\tB1\t2000-05-17\ten\tPDFEP\t1\thttps://data.epo.org/publication-server/pdf-document?cc=EP&pn=0600858&ki=B1&pd=2000-05-17'

Total Patents: 2
4/93 - 'EP\t0600865\tB1\t2000-09-20\ten\tTITLE\t1\tXYLANASE PRODUCTION\n'
5/93 - 'EP\t0600865\tB1\t2000-09-20\ten\tDESCR\t1\t<p id="p0001" num="0001">This invention lies in the field of recombinant DNA technology. '
6/93 - 'EP\t0600865\tB1\t2000-09-20\ten\tCLAIM\t1\t<claim id="c-en-01-0001" num="0001"><claim-text>A recombinant DNA material comprising a n'
7/93 - 'EP\t0600865\tB1\t2000-09-20\ten\tPDFEP\t1\thttps://data.epo.org/publication-server/pdf-doc

In [14]:
for x in patents['patent_1']:
    print(num_tokens_from_string(x))

30
4755
925
56


In [15]:
type(patents['patent_1'])

list

In [16]:
condom = patents['patent_1']
condom_dict = {}
total_claims = 1
for x in condom:
    if '\tTITLE\t' in x:
        condom_dict['title'] = x
    elif '\tDESCR\t' in x:
        condom_dict['descr'] = x
    elif '\tCLAIM\t' in x:
        condom_dict[f'claim_{total_claims}'] = x
        total_claims += 1
    elif '\tPDFEP' in x:
        condom_dict['pdfep'] = x
    else:
        raise ValueError(f"Expected sections in [TITLE, DESCR, CLAIM, PDFEP]. Received: {x}")

In [17]:
for k, v in condom_dict.items():
    print(k, num_tokens_from_string(v))

title 30
descr 4755
claim_1 925
pdfep 56


In [18]:
from typing import Dict


async def _extract_title(string: str) -> str:
    """Extract a title from `string` that is max 7 words long."""
    doctran = Doctran(
        openai_api_key=os.getenv("OPENAI_API_KEY"), openai_model="gpt-3.5-turbo"
    )
    document = doctran.parse(content=string)
    properties = ExtractProperty(
        name="title",
        description="The title of the document (max 7 words).",
        type="string",
        required=True,
    )
    document = await document.extract(properties=[properties]).execute()
    return document.transformed_content

async def divide_sections_if_too_large(
    article_dict: Dict[str, str],
    max_section_length: int = 512,
    doc_type: str = "patent",
) -> Dict[str, str]:
    """This function takes an existing dictionary containing the plan and sections
    content (from above functions), checks if any section is too large (i.e., more
    than 512 tokens), divides such sections into smaller sections, generates a new
    title, and returns the updated dictionary
    """
    if doc_type not in ["patent", "wikipedia", "arxiv"]:
        raise ValueError(
            f"doc_type must be one of 'patent', 'wikipedia', or 'arxiv'. Got {doc_type}."
        )
    logger.info("Dividing sections if too large in plan and section content.")
    final_dict: Dict = {}
    start_dict = copy(article_dict)

    def is_reference_section(heading: str):
        """Returns True if heading is a reference section."""
        heading = heading.lower()
        result = (
            "reference" in heading
            or "further reading" in heading
            or "see also" in heading
        )
        return result

    for heading, content in start_dict.items():
        num_tokens = num_tokens_from_string(content)
        # Each section must contain something, otherwise the embedding models fail
        if num_tokens == 0:
            final_dict[heading] = " "
        # If the section is small enough, add it to the final dict
        elif num_tokens <= max_section_length:
            final_dict[heading] = content
        # If section is too big, split into smaller sections, extract title, and add to final dict
        else:
            # Split
            char_splitter = RecursiveCharacterTextSplitter(
                chunk_size=max_section_length,
                chunk_overlap=0,
                # ' ' separator means sometimes sentences will be cut in two to ensure
                # the chunk size is not exceeded
                separators=["\n\n", "\n", " "],
                length_function=num_tokens_from_string,
            )
            splits: List[str] = char_splitter.split_text(content)
            # Keep heading the same but add numbers to sections e.g. 'h2 Reference' -> 'h2 Reference 1'
            if doc_type == "wikipedia" and is_reference_section(heading):
                for i, split in enumerate(splits, start=1):
                    new_heading = f"{heading} {i}"
                    final_dict[new_heading] = split
                    logger.info(
                        f"Added '{new_heading}' split original heading '{heading}'"
                    )
            # Create new titles for each split
            for split in splits:
                # Headings are of the form h1, h2, h3 etc. we split it into more of the same level
                if doc_type == "wikipedia":
                    heading_level = int(heading[1])
                    title = await _extract_title(split)
                    new_heading = f"h{heading_level} {title}"
                # Heading levels aren't important for other doc_types
                else:
                    new_heading = await _extract_title(split)
                final_dict[new_heading] = split
                logger.info(f"Added '{new_heading}' split original heading '{heading}'")

    n_keys_start = len(start_dict.keys())
    n_keys_final = len(final_dict.keys())
    logger.info(
        f"\n\tFinished dividing sections if too large in plan and section content."
        f"\n\tStarted with {n_keys_start} sections and got {n_keys_final} final sections."
        f"\n\tThat's a {n_keys_final / n_keys_start:.2f}x increase in sections"
    )
    return final_dict

In [19]:
divided = await divide_sections_if_too_large(condom_dict)

2023-08-09 17:47:22.468 | INFO     | __main__:divide_sections_if_too_large:33 - Dividing sections if too large in plan and section content.
2023-08-09 17:47:23.668 | INFO     | __main__:divide_sections_if_too_large:86 - Added 'EP 0600858 B1' split original heading 'descr'
2023-08-09 17:47:24.647 | INFO     | __main__:divide_sections_if_too_large:86 - Added 'Male Condom and Protection during Oral Sex' split original heading 'descr'
2023-08-09 17:47:25.333 | INFO     | __main__:divide_sections_if_too_large:86 - Added 'Condoms for Oral Sex' split original heading 'descr'
2023-08-09 17:47:26.246 | INFO     | __main__:divide_sections_if_too_large:86 - Added 'Condom for Oral-Genital Use' split original heading 'descr'
2023-08-09 17:47:27.079 | INFO     | __main__:divide_sections_if_too_large:86 - Added 'Embodiments of the Invention' split original heading 'descr'
2023-08-09 17:47:27.797 | INFO     | __main__:divide_sections_if_too_large:86 - Added 'Condom Design' split original heading 'desc

In [28]:
condom_dict.keys()

dict_keys(['title', 'descr', 'claim_1', 'pdfep'])

In [20]:
divided.keys()

dict_keys(['title', 'EP 0600858 B1', 'Male Condom and Protection during Oral Sex', 'Condoms for Oral Sex', 'Condom for Oral-Genital Use', 'Embodiments of the Invention', 'Condom Design', 'Condom Design for Oral Sex', 'Toroidal Surfaces', 'Tongue Extension for Performing Cunnilingus', 'Condom Patent', 'Condom Flanges', 'pdfep'])

In [28]:
list(divided.values())[1]

'EP\t0600858\tB1\t2000-05-17\ten\tDESCR\t1\t<heading id="h0001"><u><b>Background of the Invention</b></u></heading><p id="p0001" num="0001">In recent years, the appearance of the HIV virus and the impending Acquired Immune Deficiency Syndrome epidemic have created an atmosphere of fear, caution and prudence in which sexually active adults must consider protecting themselves from infection by HIV virus as well as other venereal diseases. It is generally accepted as fact that the condom provides the best protection from venereal disease and HIV virus, aside from complete sexual abstinence. Unfortunately, such a positive barrier device is, at present, the only sure or certain way to prevent transmission of all STD\'s, many of which cause great personal suffering and, in the cases of AIDS and syphilis, death.</p><p id="p0002" num="0002">Condoms are generally designed to be applied to the male member, although recent adaptations have been introduced which permit their use by women in the fo

In [24]:
divided['EP 0600858 B1']

'EP\t0600858\tB1\t2000-05-17\ten\tDESCR\t1\t<heading id="h0001"><u><b>Background of the Invention</b></u></heading><p id="p0001" num="0001">In recent years, the appearance of the HIV virus and the impending Acquired Immune Deficiency Syndrome epidemic have created an atmosphere of fear, caution and prudence in which sexually active adults must consider protecting themselves from infection by HIV virus as well as other venereal diseases. It is generally accepted as fact that the condom provides the best protection from venereal disease and HIV virus, aside from complete sexual abstinence. Unfortunately, such a positive barrier device is, at present, the only sure or certain way to prevent transmission of all STD\'s, many of which cause great personal suffering and, in the cases of AIDS and syphilis, death.</p><p id="p0002" num="0002">Condoms are generally designed to be applied to the male member, although recent adaptations have been introduced which permit their use by women in the fo

In [35]:
re.findall('</heading>', ''.join(condom))

['</heading>', '</heading>', '</heading>']

In [38]:
condom_str = ''.join(condom)
for x in condom_str.split('</heading>'):
    print(x[-125:])

CONDOM FOR ORAL-GENITAL USE
EP	0600858	B1	2000-05-17	en	DESCR	1	<heading id="h0001"><u><b>Background of the Invention</b></u>
colour to enhance enjoyment during use.<!-- EPO <DP n="5"> --></p><heading id="h0002"><u>Brief Description of the Drawing</u>
tion for cunnilingus.</li></ul><!-- EPO <DP n="7"> --></p><heading id="h0003"><u>Description of the Preferred Embodiments</u>
P	0600858	B1	2000-05-17	en	PDFEP	1	https://data.epo.org/publication-server/pdf-document?cc=EP&pn=0600858&ki=B1&pd=2000-05-17



In [34]:
type(condom)

list

In [42]:
for k, v in divided.items():
    print(k, num_tokens_from_string(v))

title 30
EP 0600858 B1 511
Male Condom and Protection during Oral Sex 512
Condoms for Oral Sex 511
Condom for Oral-Genital Use 512
Embodiments of the Invention 512
Condom Design 513
Condom Design for Oral Sex 512
Toroidal Surfaces 512
Tongue Extension for Performing Cunnilingus 151
Condom Patent 512
Condom Flanges 413
pdfep 56
